<a href="https://colab.research.google.com/github/meprateekpaul/DataMining/blob/main/PubMedMineR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is created by Prateek Paul.
* Email: prateekp@iiitd.ac.in
* LinkedIn: [linkedin.com/in/prateekpaulpro/](https://linkedin.com/in/prateekpaulpro/)

Disclaimer:
The code and content in this notebook are compiled from various open sources, personal experience, and reference materials. It is intended solely for educational purposes. All credits for original ideas and code snippets go to their respective authors. If you find any inaccuracies or have suggestions, feel free to reach out.

# INSTALLATIONS

In [ ]:
#install.packages('rcrossref')
install.packages('pubmed.mineR')
#library(rcrossref)
library(pubmed.mineR)
# 9mins install time

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘RCurl’, ‘XML’, ‘R2HTML’, ‘RJSONIO’




In [ ]:
install.packages('RISmed')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
#library(BiocParallel)
library(RISmed)

# SEARCH

In [ ]:
#title Linking
email <- '18btbinf003@shiats.edu.in'
api_key <- "6907d480080e385d850bc63d2d569cff2708"
system("api_key=6907d480080e385d850bc63d2d569cff2708")

In [ ]:
term <- "phytochemical" #@param {type:"string"}
#retake <- 10000 #@param {type:"integer"}
min_date <- 2000 #@param {type:"integer"}

res1 <- EUtilsSummary(term,
                      type = "esearch",
                      db = "pubmed",
                      datetype = "pdat",
 #                     retmax = retake,
                      mindate = min_date,
                      maxdate = 2023  # Set a maximum date if needed
)

fetch <- EUtilsGet(res1, type = 'efetch', db = "pubmed")
abstracts <- data.frame(title = fetch@ArticleTitle,
                        abstract = fetch@AbstractText,
                        journal = fetch@Title,
                        PMID = fetch@PMID,
                        year = fetch@YearPubmed)

## Ensure abstracts are character fields (not factors)
abstracts <- abstracts %>% mutate(abstract = as.character(abstract))

In [ ]:
abstracts <- read.csv('/content/csv-probeORtra-set.csv')
length(abstracts$PMID)
pmids_new=unique(abstracts$PMID)
length(pmids_new)

[1] 10000

[1] 10000

In [ ]:
# Getting abstracts
#abstracts.Abstracts = lapply(pmids_new, pubtator_function)

# Basics Mining of PubTAtOR

In [ ]:
res2 = lapply(pmids_new, pubtator_function)
#res2 = bplapply(pmids_new, pubtator_function)
df <-  as.data.frame(do.call(rbind, res2))
df <- apply(df,2,as.character)

In [ ]:
df

In [ ]:
write.csv(df,"heartattac.csv", row.names = FALSE)

In [ ]:
finaldf<-merge(x = df, y = abstracts, by = 'PMID', all = TRUE)
write.csv(finaldf,"heartattac.csv", row.names = FALSE)

In [ ]:
pubtator_result_list_to_table(res2)
pubresults <- read.csv('result.txt', sep='\t')

# Seperating Terms and Numbers

In [ ]:
df_entities <- data.frame(PMID=character(),PubTract=character(),PubType=character())
for (x in list("Genes","Chemicals","Species","Mutations","Diseases")){
#for (x in list("Chemicals")){
#for (x in list("Genes","Chemicals","Species")){
  for (y in 1:length(res2)){
    for (z in try(res2[[y]][[x]])){
      try(df_entities <- rbind(df_entities, data.frame(PMID=res2[[y]][["PMID"]],PubTract=z, PubType=x)))
  }}}
dim(df_entities)

In [ ]:
write.csv(df_entities,"heartattacTypes.csv", row.names = FALSE)

In [ ]:
install.packages('rcrossref')
library(rcrossref)
library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘triebeard’, ‘lazyeval’, ‘urltools’, ‘httpcode’, ‘Rcpp’, ‘httpuv’, ‘xtable’, ‘sourcetools’, ‘later’, ‘promises’, ‘htmlwidgets’, ‘crosstalk’, ‘crul’, ‘plyr’, ‘shiny’, ‘miniUI’, ‘DT’


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
#getting Pubmed IDs
df_pmc <- data.frame(PMID=character(),PMC=character())
for (x in pmids_new){
  ids <- id_converter(x, "pmid")
  pmc = str_replace(ids$records$pmcid[1], "PMC", "")
  try(df_pmc <- rbind(df_pmc, data.frame(PMID=x,PMC=pmc)))
}

In [ ]:
df_entities <- separate(data = df_entities, col = PubTract, into = c("PName", "PMeshId"), sep = ">")
df_pmc_ent <-merge(x = df_entities, y = df_pmc, by = 'PMID', all = TRUE)

Warning message:
“Expected 2 pieces. Additional pieces discarded in 1 rows [25838].”


In [ ]:
df_pmc_ent <- read.csv('df_pmc_ent.csv')

In [ ]:
df_pmc_ent

In [ ]:
df_chem <- df_pmc_ent[df_pmc_ent$PubType == 'Chemicals',]

In [ ]:
#sentences <- bplapply(df_pmc_ent$PMC, Give_Sentences_PMC, term=df_pmc_ent$PName)
sentences <- bplapply(df_chem$PMC, Give_Sentences_PMC, term=df_chem$PName)

In [ ]:
dim(sentences)

NULL

In [ ]:
length(df_chem$PMC)

[1] 438

In [ ]:
length(sentences[263])

[1] 1

In [ ]:
df_sentences <- rbind(df_chem, sentences)

In [ ]:
rownames(df_chem) <- 1:nrow(df_chem)

In [ ]:
df_chem$sentences <- sentences

In [ ]:
df_chem

,PMID,PName,PMeshId,PubType,PMC,sentences
,<int>,<chr>,<chr>,<chr>,<int>,<list>
56,17406030,temozolomide,MESH:D000077204,Chemicals,NA,NULL
68,17620414,H3-K36-Me,-,Chemicals,2099615,NULL
70,17620414,H3-K4-Me,-,Chemicals,2099615,NULL
72,17620414,ATP,MESH:D000255,Chemicals,2099615,NULL
103,18794128,oncrasin-1,MESH:C532252,Chemicals,2678915,NULL
118,19074863,carboplatin,MESH:D016190,Chemicals,NA,NULL
123,19144191,sphingolipid,MESH:D013107,Chemicals,2632640,NULL
124,19144191,dihydromotuporamine C,MESH:C510298,Chemicals,2632640,NULL
143,19366795,estradiol,MESH:D004958,Chemicals,2811393,NULL


In [ ]:
df_chem

PMID,PName,PMeshId,PubType,PMC,sentences
17406030,temozolomide,MESH:D000077204,Chemicals,NA,NULL
17620414,H3-K36-Me,-,Chemicals,2099615,NULL
17620414,H3-K4-Me,-,Chemicals,2099615,NULL
17620414,ATP,MESH:D000255,Chemicals,2099615,NULL
18794128,oncrasin-1,MESH:C532252,Chemicals,2678915,NULL
19074863,carboplatin,MESH:D016190,Chemicals,NA,NULL
19144191,sphingolipid,MESH:D013107,Chemicals,2632640,NULL
19144191,dihydromotuporamine C,MESH:C510298,Chemicals,2632640,NULL
19366795,estradiol,MESH:D004958,Chemicals,2811393,NULL
19366795,BEZ235,MESH:C531198,Chemicals,2811393,NULL


In [ ]:
df_chem <- apply(df_chem,2,as.character)
write.csv(df_chem, "df_chem.csv", row.names=FALSE)

# Getting PMC

In [ ]:
Give_Sentences_PMC(df_pmc_ent$PMC[1], df_pmc_ent$PName[1])

[1] " In vivo depletion of Lcp5p also inhibited pre-rRNA processing."                                                                                                                    
[2] " Indirect immunofluorescence indicated a predominant localization of Lcp5p in the nucleolus."                                                                                       
[3] " In addition, antibodies directed against Lcp5p specifically immunoprecipitated the yeast U3 snoRNA snR17, suggesting that the protein is directly involved in pre-rRNA processing."

In [ ]:
Give_Sentences_PMC(PMC4039032, "atherosclerosis")

# counts


In [ ]:
pubresults[['Chemicals','PMID']]

ERROR: ignored

In [ ]:
df_chem <- df_pmc_ent[df_pmc_ent$PubType == 'Chemicals',]